In [12]:
%pip -q install google-genai

In [14]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [15]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [16]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [17]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [18]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [19]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [30]:
################################################
# --- Agente 1: Buscador de Plantas Baixas --- #
################################################
def agente_buscador(especificacao):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17-thinking",
        instruction="""
        Você é um assistente de pesquisa. Sua tarefa é usar a ferramenta de busca do google (google_search) para recuperar descrições de plantas baixas seguindo as especificacoes abaixo.
        Procure chegar o mais perto possivel de atender à especificacao
        Foque nas 5 plantas mais próximas da especificação.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Especificação:{especificacao}"

    plantas = call_agent(buscador, entrada_do_agente_buscador)
    return plantas

In [31]:
################################################
# --- Agente 2: Julgador de plantas --- #
################################################
def agente_julgador(especificacao, plantas_buscadas):
    julgador = Agent(
        name="agente_julgador",
        model="gemini-2.5-flash-preview-04-17-thinking",
        # Inserir as instruções do Agente Julgador #################################################
        instruction="""
        Você é um julgador de plantas baixas, especialista projeto de casas. Com base na lista de plantas próximas da especificação, você deve
        eleger a descrição de planta baixa que melhor se adequa à especificação.
        Apresente uma justificativa para sua escolha.
        """,
        description="Agente que julga a melhor planta baixa",
        tools=[google_search]
    )

    entrada_do_agente_julgador = f"Especificação:{especificacao}\nPlantas buscadas: {plantas_buscadas}"
    # Executa o agente
    melhor_planta = call_agent(julgador, entrada_do_agente_julgador)
    return melhor_planta

In [22]:
######################################
# --- Agente 3: Desenhista --- #
######################################
def agente_desenhista(melhor_planta):
    desenhista = Agent(
        name="agente_desenhista",
        model="imagen-3.0-generate-002",
        instruction="""
            Você é um desenhista de plantas baixas e irá seguir a descrição abaixo para gerar uma imagem de planta baixa em preto e branco.
            """,
        description="Agente desenhista de planta baixa"
    )

    entrada_do_agente_desenhista = f"Planta baixa a ser gerada: {melhor_planta}"
    # Executa o agente
    rascunho = call_agent(desenhista, entrada_do_agente_desenhista)
    return rascunho

In [32]:
print("🚀 Iniciando o Sistema de Criação de planta baixa de casa com 3 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
especificacao = input("❓ Especifique sua casa dos sonhos (E.g. 200m2 com 2 suites, sala, cozinha, garagem, etc): ")

# Inserir lógica do sistema de agentes ################################################
if not especificacao:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar uma planta baixa de uma casa com {especificacao}")

    plantas = agente_buscador(especificacao)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(plantas))
    print("--------------------------------------------------------------")

    melhor_planta = agente_julgador(especificacao, plantas)
    print("\n--- 📝 Resultado do Agente 2 (Julgador) ---\n")
    display(to_markdown(melhor_planta))
    print("--------------------------------------------------------------")


#    imagem_planta = agente_desenhista(melhor_planta)
#    print("\n--- 📝 Resultado do Agente 3 (Desenhista) ---\n")
#    display(to_markdown(imagem_planta))
#    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de planta baixa de casa com 3 Agentes 🚀
❓ Especifique sua casa dos sonhos (E.g. 200m2 com 2 suites, sala, cozinha, garagem, etc): 200m2 com 2 suites, sala, cozinha, garagem
Maravilha! Vamos então criar uma planta baixa de uma casa com 200m2 com 2 suites, sala, cozinha, garagem

--- 📝 Resultado do Agente 1 (Buscador) ---



> Com base na sua especificação de uma planta baixa de aproximadamente 200m² com 2 suítes, sala, cozinha e garagem, encontrei as seguintes opções que se aproximam ou atendem a esses critérios:
> 
> 1.  **Planta de casa térrea com 1 quarto, 2 suítes e varanda gourmet - Cód. 180 (176 m²)**: Embora liste 1 quarto e 2 suítes, totalizando 3 dormitórios, a área construída de 176m² está próxima dos 200m². Inclui 3 banheiros (o que se alinha com 2 suítes mais um lavabo ou banheiro social) e varanda gourmet. A descrição completa não detalha a sala e cozinha separadamente, mas é comum em projetos com varanda gourmet e múltiplos quartos/suítes terem esses ambientes. A garagem não é explicitamente mencionada no resumo, mas o site de origem possui filtros para garagem.
> 2.  **Projeto de Casa Térrea Com 2 Suítes (Preço do Projeto: R$ 1.190,00)**: Este projeto é explicitamente nomeado como tendo 2 suítes. A área não é especificada no resumo inicial, mas está listado sob a categoria de plantas de casas térreas com área construída de até 200m². O site oferece a planta baixa com medidas e imagens ilustrativas da fachada. A descrição não detalha sala, cozinha e garagem no snippet, mas a categoria principal abrange plantas com área de até 200m².
> 3.  **Planta de casa com 2 suites e 1 quarto (147 m²)**: Esta opção possui 2 suítes e mais 1 quarto, totalizando 3 dormitórios. A área de 147m² é um pouco menor que a especificação, mas inclui sala de TV, sala de jantar e cozinha integradas, além de garagem para 2 carros.
> 4.  **Projeto de casa com 2 suítes | G05A (159.30 m²)**: Este é um projeto de sobrado com 2 suítes, sala de estar, sala de jantar e cozinha integradas, e área de serviço. Possui 4 banheiros no total e 1 vaga de garagem. A área construída é de 159.30m².
> 5.  **Casa com 2 quartos sendo um do tipo suíte (200,00 m² de área construída)**: Encontrada em um anúncio de venda, esta casa possui 2 quartos, sendo 1 suíte. A área construída é exatamente 200m². Inclui armários planejados na cozinha, área gourmet com churrasqueira, piscina e garagem para 6 carros (sendo 2 cobertos). Embora tenha apenas 1 suíte, a área e outros cômodos se encaixam na busca.
> 
> É importante notar que algumas das descrições encontradas incluem 3 quartos (sendo 2 suítes), ou áreas próximas a 200m² com 2 suítes e outros cômodos que podem ser adaptados. Alguns sites permitem filtrar por número de suítes, quartos, área construída e inclusão de garagem.
> 
> Para ter acesso à planta baixa detalhada e visualizar o layout completo desses projetos, geralmente é necessário visitar os sites de origem indicados nos resultados da busca.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Julgador) ---



> Como julgador de plantas baixas, com base na sua especificação de uma planta de aproximadamente 200m² com 2 suítes, sala, cozinha e garagem, avaliei as opções apresentadas.
> 
> A planta baixa que melhor se adequa à sua especificação é a de número 2: **Projeto de Casa Térrea Com 2 Suítes (Preço do Projeto: R$ 1.190,00)**.
> 
> **Justificativa:**
> 
> Esta opção é a única que explicitamente menciona possuir **2 suítes** no seu título e descrição resumida, atendendo diretamente a um dos requisitos cruciais da sua especificação. Embora a área não seja especificada como exatos 200m², ela está listada sob a categoria de plantas de casas térreas com área construída **de até 200m²**, o que a torna a mais próxima em termos de área dentro das opções que atendem ao número de suítes.
> 
> As outras opções apresentam desvios mais significativos:
> *   A opção 1 possui 2 suítes, mas totaliza 3 dormitórios e a área (176m²) é um pouco abaixo do especificado.
> *   As opções 3 e 4, apesar de mencionarem 2 suítes, possuem áreas consideravelmente menores (147m² e 159.30m²) do que os 200m² desejados. Além disso, a opção 3 também tem 3 dormitórios no total e a opção 4 é um sobrado.
> *   A opção 5 tem a área exata de 200m², mas possui apenas 1 suíte, não atendendo ao requisito fundamental de 2 suítes.
> 
> Apesar da descrição da opção 2 não detalhar explicitamente sala, cozinha e garagem no resumo fornecido, é altamente provável que um "Projeto de Casa Térrea Com 2 Suítes" dentro da categoria de até 200m² inclua esses ambientes essenciais em uma residência padrão. A especificação clara das 2 suítes e a adequação da área a tornam a candidata mais promissora a atender à sua necessidade geral.
> 
> Para confirmar completamente se atende a todos os requisitos (sala, cozinha e garagem), seria necessário consultar a planta baixa detalhada no site de origem. No entanto, com base nas informações fornecidas, a opção 2 é a que melhor se alinha com a sua busca por uma planta com 2 suítes e aproximadamente 200m².


--------------------------------------------------------------
